### Fera formidável 4.6 - E se meus dados forem imagens?

#### Enunciado

Objetivo: implementar uma rede neural convolucional (CNN) utilizando PyTorch ou
lightning. Treine esta rede neural em um conjunto de dados de imagens. Explique para
o leitor como funciona a camada de convolução de uma CNN e o motivo de utilizarmos
este tipo de arquitetura quando estudamos imagens.

Dica: um dos mais famosos conjuntos de dados de imagens é o MNIST.

In [33]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets  
import torchvision.transforms as transforms
import time

In [34]:
class CNN(nn.Module):
    def __init__(self): 
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # Primeira camada convolucional
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # Segunda camada convolucional   
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # Camada de agrupamento máximo que reduz as dimensões de largura e altura pela metade
        self.fc1 = nn.Linear(7*7*64, 128) # Camada totalmente conectada 1
        self.fc2 = nn.Linear(128, 10) # Camada totalmente conectada 2

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 7*7*64)
        x = F.relu(self.fc1(x))

        return x

In [40]:
BATCH_SIZE = 64 # Número de amostras de imagens processadas ao mesmo tempo a cada iteração

# Carregando dados de treino train= True
df_treino = datasets.MNIST(root= 'dataset/' , train= True , transform=transforms.ToTensor(), download= True ) 
loader_treino = DataLoader(dataset=df_treino, batch_size=BATCH_SIZE, shuffle= True ) 

# Carregando dados de teste train= False
df_teste = datasets.MNIST(root= 'dataset/' , train= False , transform=transforms.ToTensor(), download= True ) 
loader_teste = DataLoader(dataset=df_teste, batch_size=BATCH_SIZE, shuffle= True )

In [35]:
# Apesar de não ajudar no caso de GPU integrada achei interessante manter essa linha

device = torch.device( 'cuda'  if torch.cuda.is_available() else  'cpu' )
device

device(type='cpu')

In [36]:
TAXA_APRENDIZADO = 0.001
NUM_EPOCAS = 10 # A cada época a rede neural terá passado por todas as imagens do dataset

modelo = CNN().to(device)

In [37]:
perda = nn.CrossEntropyLoss() # Mede a dissimilaridade entra a distribuição de rótulos verdadeira e as previsões do modelo
otimizador = optim.Adam(modelo.parameters(), lr= TAXA_APRENDIZADO)

In [38]:
inicio = time.time()
for epoca in range(NUM_EPOCAS):
    for images, labels in loader_treino:
        outputs = modelo(images)
        loss = perda(outputs, labels)
        otimizador.zero_grad()
        loss.backward()
        otimizador.step()
        
    print(f"Época {epoca + 1} concluída")

fim = time.time()
print(f"Treinamento finalizado após {(fim - inicio) / 60} minutos")

Época: 1 concluída
Época: 2 concluída
Época: 3 concluída
Época: 4 concluída
Época: 5 concluída
Época: 6 concluída
Época: 7 concluída
Época: 8 concluída
Época: 9 concluída
Época: 10 concluída
Treinamento finalizado após 8.317753819624583 minutos


In [41]:
# Loop de validação
num_acertos = 0
num_total = 0

for imagens, rotulos in loader_teste:
    imagens = imagens.to(device)
    rotulos = rotulos.to(device)

    saida = modelo(imagens)                      # Passa as imagens pela CNN
    previsoes = torch.argmax(saida, dim=1)       # Pega a classe com maior probabilidade

    acertos = (previsoes == rotulos).sum().item()  # Conta quantos acertaram
    num_acertos += acertos
    num_total += rotulos.size(0)                  # Soma quantas imagens tinham no batch

# Depois do loop:
acuracia = num_acertos / num_total
print(f'Acurácia: {acuracia * 100:.2f}%')


Acurácia: 59.60%


### Referências

BHARGAVI POYEKAR. Building simple Neural Networks using Pytorch (NN, CNN) for MNIST dataset. Disponível em: <https://medium.com/%40bpoyeka1/building-simple-neural-networks-nn-cnn-using-pytorch-for-mnist-dataset-31e459d17788>. Acesso em: 2 maio. 2025.